In [2]:
# Import and Install
%pip install -r requirements.txt --upgrade
import fooddatacentral as fdc
import pandas as pd
import numpy as np
from datascience import *
from scipy.optimize import linprog

apikey = "hhXDThEWfQoAua41aVdG81iaAHEhLCC0An6FqziG"

  Using cached pint-0.25.2-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-2.4.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached pandas-3.0.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (79 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.17.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached gspread-6.2.1-py3-none-any.whl.metadata (11 kB)
  Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached bottleneck-1.6.0-cp311-cp311-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (8.2 kB)
  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl.metadata (363 bytes)
  Using cached fooddatacentral-1.0.12-py3-none-any.whl.metadata (2.1 kB)
  Using cached python_gnupg-0.5.6-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached flexcache-0.3-py3-none-any.whl.metadata (7.0 kB)
  Using

In [3]:
# [A] Description of population interest

# Reading Table
df_min = Table.read_table('Dietary Requirements - diet_minimums.csv')
df_max = Table.read_table('Dietary Requirements - diet_maximums.csv')

#Sex: Type F or M
def dri(age, sex):
    if age <= 3:
        sex = 'C'
        age_t = '1-3'
    elif 4 <= age <= 8:
        age_t = '4-8'
    elif 9 <= age <= 13:
        age_t = '9-13'
    elif 14 <= age <= 18:
        age_t = '14-18'
    elif 19 <= age <= 30:
        age_t = '19-30'
    elif 31 <= age <= 50:
        age_t = '31-50'
    else:
        age_t = '51+'

    column = sex + " " + age_t

    # Some nutrients does not have a maximum so it should be infinite
    # Using 999999 because later linprog function does not allow float('Inf')
    # But 999999 would be fair as it is a really big number
    data = {
            'Nutrition': np.insert(df_min.column("Nutrition"), 0, 'Sodium, Na') ,
            'max': np.append(df_max.column(column), [999999]*(len(df_min.column(column))-1)),
            'min': np.insert(df_min.column(column), 0, 0) 
            }

    return pd.DataFrame(data)


restrictions = dri(20, 'F')
restrictions

,Nutrition,max,min
0,"Sodium, Na",2300,0.0
1,Energy,3100,2000.0
2,Protein,999999,46.0
3,"Fiber, total dietary",999999,28.0
4,"Folate, DFE",999999,400.0
5,"Calcium, Ca",999999,1000.0
6,"Carbohydrate, by difference",999999,130.0
7,"Iron, Fe",999999,18.0
8,"Magnesium, Mg",999999,310.0
9,Niacin,999999,14.0


In [4]:
# [A] Nutritional content of different foods

# Put an FDC ID HERE!
id =  [['chicken',2727569],
       ['rice', 2512381]] 

def nutrition_table(id):
    nutrient_df = pd.DataFrame({
                    'Nutrition': np.insert(df_min.column("Nutrition"), 0, 'Sodium, Na') ,
                    })
    
    for i in id:
        # Modifying data
        food_nutrient = fdc.nutrients(apikey,fdc_id=i[1])
        food_nutrient = food_nutrient.rename(index={"Energy (Atwater General Factors)": "Energy"})
        food_nutrient = food_nutrient.reset_index().rename(columns={'index': 'Nutrition','Quantity':i[0]}).drop("Units", axis=1)
        food_nutrient[food_nutrient.select_dtypes("number").columns] = food_nutrient.select_dtypes("number").clip(lower=0)
        
        
        nutrient_df = pd.merge(nutrient_df, food_nutrient, on='Nutrition', how='left')
        
    return nutrient_df.fillna(0)

nutrition_table(id)

,Nutrition,chicken,rice
0,"Sodium, Na",48.07000,0.46250
1,Energy,126.90500,358.70500
2,Protein,21.40625,7.03885
3,"Fiber, total dietary",0.00000,0.14880
4,"Folate, DFE",0.00000,0.00000
5,"Calcium, Ca",6.89800,4.46000
6,"Carbohydrate, by difference",0.00000,80.31315
7,"Iron, Fe",0.35350,0.14050
8,"Magnesium, Mg",25.40000,26.53000
9,Niacin,0.00000,1.43400


In [47]:
A_ub = []
for i in range(len(nutrition_table(id))):
    A_ub.append(list(nutrition_table(id).iloc[i][1:]))

for i in range(len(A_ub)):
    negative_a_ub = []
    for j in range(len(A_ub[0])):
        negative_a_ub.append(-A_ub[i][j])
    A_ub.append(negative_a_ub)
    
A_ub

[[48.07, 0.46250000000000002],
 [126.905, 358.70499999999998],
 [21.40625, 7.0388500000000001],
 [0.0, 0.14879999999999999],
 [0.0, 0.0],
 [6.8979999999999997, 4.46],
 [-0.42825000000000002, 80.313149999999993],
 [0.35349999999999998, 0.14050000000000001],
 [25.399999999999999, 26.530000000000001],
 [0.0, 1.4339999999999999],
 [203.59999999999999, 108.2],
 [332.5, 82.260000000000005],
 [0.0, 0.080000000000000002],
 [0.0, 0.065000000000000002],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.057880000000000001],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.62450000000000006, 1.3540000000000001],
 [-48.07, -0.46250000000000002],
 [-126.905, -358.70499999999998],
 [-21.40625, -7.0388500000000001],
 [-0.0, -0.14879999999999999],
 [-0.0, -0.0],
 [-6.8979999999999997, -4.46],
 [0.42825000000000002, -80.313149999999993],
 [-0.35349999999999998, -0.14050000000000001],
 [-25.399999999999999, -26.530000000000001],
 [-0.0, -1.4339999999999999],
 [-203.59999999999999, -108.2],
 [-332.5, -82.260000000000005],
 [-

In [51]:
# [A] Solution

df_nutrition = Table.read_table('Dietary Food List - Sheet4.csv')

c = [6]*14
A_ub = []
for i in range(len(df_nutrition[0])):
    A_ub.append(list(df_nutrition.row(i))[2:])
    
for i in range(len(A_ub)):
    negative_a_ub = []
    for j in range(len(A_ub[0])):
        negative_a_ub.append(-A_ub[i][j])
    A_ub.append(negative_a_ub)

b_ub = list(restrictions["max"]) + list(-restrictions["min"])

bounds = []
for i in range(len(A_ub[0])):
    bounds.append((0, None))

solution = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)
solution.x

list_of_food = list(df_nutrition.labels)[2:]
list_of_food  
df = pd.DataFrame({'Food': list_of_food,
        '100g': solution.x})
df

,Food,100g
0,Cabbage,1.499154
1,Mango,0.086810
2,Chicken,0.000000
3,Rice,0.000000
4,Soybean Oil,1.205168
5,Bread,0.196292
6,Onion,0.000000
7,Raw Suagr,0.000000
8,Beans (Black),1.831514
9,Fish/Tuna,1.048087
